# Create & Use Log Functions

While developers can use a given logger object directly for logging, the {{poptus}} logging interface includes the high-level ``create_log_functions`` function for creating a set of different logging functions built off of the same logger object.  In this example, we explore the use of this function as well as the log functions

* ``log(msg)`` - register the given general information message with the logger with the ``LOG_LEVEL_DEFAULT`` level for possible logging
* ``log_debug(msg, debug_level)`` - register the given debug message and its given verbosity level ``debug_level`` with the logger for possible logging
* ``warn(msg)`` - log the given warning message
* ``log_and_abort(my_exception, msg)`` - log the given error message and then raise the given exception ({{eg}} ``ValueError``, ``TypeError``, ``RuntimeError``, {{etc}})

generated by it.  Since this portion of the logging interface is the minimal amount of information that a developer needs to use logging in their code, the example does not explore configuring or creating logger objects.  Please refer to the "User" portion of this book to find examples related with those topics.

This example shows a single "method" function that has been developed using the high-level logging interface and that doesn't do anything interesting beyond logging.  For this silly, lil' method, we impose the following criteria for determining what information should be written with which debug log levels

* `LOG_LEVEL_MIN_DEBUG` - Insights for tracing method's progress.  Potentially useful for careful users and method developers.  This should not be an overwhelming amount of information.
* `LOG_LEVEL_MIN_DEBUG` + 1 - Reporting coordinates of a parameter-space point.  For high-dimensional spaces, such information could be substantially long and could render the log ineffective for general use.  Users might find this useful for checking setups.
* `LOG_LEVEL_MIN_DEBUG` + 2 - Reporting internal and intermediate data for helping developers develop and debug the method.

The method has been developed with 

* a low-level implementation that applies the method to given input and
* a high-level implementation that wraps the low-level implementation in a simpler, user-friendly interface.

While these two layers could be combined into a single function, this design allows users to use the implementation that suits their needs.  Typically the low-level implementation would be used by expert users; the high-level, by nonexperts who can benefit from a bit of help from the developers.

## Low-level implementation
Method and model codes are often long and mathematically complex codes.  As such, they can be difficult to develop and maintain, which motivates the design decision to keep the code's implementation small and as simple as possible.  In that spirit, we are imagining that this function should accept

* a complete list of all arguments needed to fully configure and control the method for all use cases,
* all points ({{eg}} parameter-space points) as ordered numpy arrays with no mapping of problem-specific names onto coordinates, and
* a valid {{poptus}} logger object.

As a result, this interface requires explicit programming by insisting that users make a full set of decisions for how to use the method, including the configuration of their logger, and record these in the form of the method's argument list.

Note that by accepting a logger object and using only `create_log_functions` our code is strongly decoupled from the {{poptus}} package.  We only need to know that a logger class exists and how to use the above set of generated functions.  In particular, it doesn't need to know what type of logging can be done, where logging can be recorded, or how to configure the logging facilities to achieve this.  In addition, it doesn't need to know if there is a {{poptus}}-wide default logger or how to create it.

In [1]:
# Remove input/outputs in rendering

import sys

from pathlib import Path

# We also assume that this is in the path below when loading the model function.
sys.path.append(Path.cwd())

from print_file import print_file

In [2]:
import poptus

import numpy as np


def low_level_method(x_0, max_iters, stop_tolr, expert_factor, logger):
    # -- HARDCODED VALUES
    LOG_NAME = "MyMethod"

    DEBUG_0    = poptus.LOG_LEVEL_MIN_DEBUG
    DEBUG_LONG = poptus.LOG_LEVEL_MIN_DEBUG + 1
    DEBUG_DEV  = poptus.LOG_LEVEL_MIN_DEBUG + 2

    LARGE_STOP_TOLR = 1.0e-3

    # -- CREATE FUNCTIONS FOR ALL LOGGING
    log, log_debug, warn, log_and_abort = \
        poptus.create_log_functions(logger, LOG_NAME)

    # -- EXTRACT & LOG ALL CONFIGURATIONS
    log("")
    log("Method Configuration")
    log("-" * 80)
    log(f"{'N Max Iterations':30}{max_iters}")
    log(f"{'Stopping Tolerance':30}{stop_tolr}")
    log_debug(f"Expert Configurations", DEBUG_0)
    log_debug(f"\t{'Factor':25}{expert_factor}", DEBUG_0)
    log_debug("", DEBUG_LONG)
    log_debug("Starting Point", DEBUG_LONG)
    log_debug("-" * 60, DEBUG_LONG)
    for i, coordinate in enumerate(x_0):
        e_i = f"e_{i+1}"
        log_debug(f"\t{e_i:<10}{coordinate}", DEBUG_LONG)
    log("")

    # -- ERROR CHECK ONLY AFTER LOGGING
    if max_iters <= 0:
        log_and_abort(ValueError, "max_iters must be positive integer")

    if stop_tolr <= 0.0:
        log_and_abort(ValueError, "stop_tolr must be positive")
    elif stop_tolr > LARGE_STOP_TOLR:
        warn("Stopping tolerance is suspiciously large")

    if (expert_factor <= 0.0) or (expert_factor >= 1.0):
        log_and_abort(ValueError, "Invalid factor")

    # -- EXECUTE OUR SILLY, LIL' ALGORITHM
    x_i = x_0.copy()
    for i in range(1, max_iters+1):
        f_i = np.linalg.norm(x_i, 2)

        # Log something at each iteration based on log level
        if logger.level < poptus.LOG_LEVEL_MIN_DEBUG:
            log(f"Iteration {i}")
        else:
            log_debug(f"f_{i} = {f_i} at {x_i}", DEBUG_0)

        if f_i <= stop_tolr:
            break

        x_i *= expert_factor
        log_debug(f"Scaling current point by {expert_factor}", DEBUG_DEV)

    if f_i > stop_tolr:
        log_and_abort(RuntimeError, "Failed to converge within allotted budget")

    log(f"Approximated Solution = {f_i}")

    return x_i, f_i

As part of our explicit, detailed invocation of this method, we presently choose to use the {{poptus}}-wide default logger, which logs to stdout/stderr with no logging of debug information.

In [3]:
x_hat, f_hat = low_level_method(
    x_0=np.array([1.1, -2.2]),
    max_iters=20,
    stop_tolr=1.0e-5,
    expert_factor=0.5,
    logger=poptus.create_logger(configuration=None)
)

print()
print(f"Solution at {x_hat}")

[MyMethod] 
[MyMethod] Method Configuration
[MyMethod] --------------------------------------------------------------------------------
[MyMethod] N Max Iterations              20
[MyMethod] Stopping Tolerance            1e-05
[MyMethod] 
[MyMethod] Iteration 1
[MyMethod] Iteration 2
[MyMethod] Iteration 3
[MyMethod] Iteration 4
[MyMethod] Iteration 5
[MyMethod] Iteration 6
[MyMethod] Iteration 7
[MyMethod] Iteration 8
[MyMethod] Iteration 9
[MyMethod] Iteration 10
[MyMethod] Iteration 11
[MyMethod] Iteration 12
[MyMethod] Iteration 13
[MyMethod] Iteration 14
[MyMethod] Iteration 15
[MyMethod] Iteration 16
[MyMethod] Iteration 17
[MyMethod] Iteration 18
[MyMethod] Iteration 19
[MyMethod] Approximated Solution = 9.38291463947208e-06

Solution at [ 4.19616699e-06 -8.39233398e-06]


## High-level interface

Since the low-level interface can be quite long and complicated, we choose to provide an optional wrapper interface around the low-level implementation geared toward regular, nonexpert users.  Some goals for designing this high-level wrapper are

* simpler interface that presents a lower bar of entry for nonexperts
* include necessary boiler plate code that would unnecessarily clutter the low-level implementation
* create the logger from user-provided configuration information
* create the correct {{poptus}}-wide default logger if no configuration information is provided
* allow users to specify and receive all points as unordered ``dict``s or pandas DataFrames that map problem-specific names to coordinates

In order to allow this wrapper to convert points between ``dict``s and numpy arrays, users need to specify an ordering for the arrays.  In the case of a parameter space, we are effectively asking them to define the basis of their parameter space.  While in many cases this ordering is arbitrary, let's be explicit and define our spaces.

In [4]:
# The parameter names alpha and beta are presumably informative for our user
PARAMETER_ORDER = ["beta", "alpha"]

In this particular case, the only other argument that our high-level wrapper requires is the name of a JSON-format configuration file whose contents it loads on behalf of the user.

Note that if some of the low-level implementation's arguments are expert only and have sensible default values that nonexpert users can blindly use, this wrapper could be encoded with those default values so that they need not be included in the JSON file.  For example, the JSON configuration file contains an optional ``expert`` section and this wrapper always uses expert configuration values when provided and default values otherwise.

In [5]:
import copy

import pandas as pd


def run_method(filename_json, parameter_order):
    # -- HARDCODED VALUES
    LOG_NAME = "MyMethod"

    DEBUG_0    = poptus.LOG_LEVEL_MIN_DEBUG
    DEBUG_LONG = poptus.LOG_LEVEL_MIN_DEBUG + 1
    DEBUG_DEV  = poptus.LOG_LEVEL_MIN_DEBUG + 2

    # Nonexpert users can blindly use these values
    DEFAULT_EXPERT_ARGS = {"factor": 0.5}

    # -- EXTRACT CONFIGURATION VALUES
    with open(filename_json, "r") as fptr:
        configuration = json.load(fptr)

    # -- CREATE FUNCTIONS FOR ALL LOGGING
    logger = poptus.create_logger(configuration["logging"])
    log, log_debug, warn, log_and_abort = \
        poptus.create_log_functions(logger, LOG_NAME)

    log(f"Configuration loaded from {filename_json}")

    # -- LOG PARAMETER SPACE DEFINITION
    # The low-level method does not have access to this info, so we must log here
    log_debug("", DEBUG_LONG)
    log_debug("Parameter-space Basis", DEBUG_LONG)
    log_debug("-" * 80, DEBUG_LONG)
    for i, parameter in enumerate(parameter_order):
        e_i = f"e_{i+1}"
        log_debug(f"{e_i:<10}{parameter}", DEBUG_LONG)

    # Assume that these are error checked and logged by low-level function
    x_0 = configuration["starting_point"]
    max_iters = configuration["max_iters"]
    stop_tolr = configuration["stopping_criterion"]

    # Manage & load optional configuration values
    expert_args = copy.deepcopy(DEFAULT_EXPERT_ARGS)
    if "expert" in configuration:
        for key, value in configuration["expert"].items():
            expert_args[key] = value

    expert_factor = expert_args["factor"]

    # -- CONVERT UNORDERED OBJECTS TO ORDERED
    # Users provide points as dicts with paramter names mapped onto coordinates.
    #
    # Methods don't know or care about that mapping, so we give them the point
    # as an ordered array using the provided ordering, which should be used
    # consistently throughout.
    x_0_ordered = np.array([x_0[p] for p in parameter_order])

    # -- LET LOW-LEVEL METHOD DO ALL THE WORK
    x_hat, f_hat = low_level_method(
        x_0=x_0_ordered,
        max_iters=max_iters,
        stop_tolr=stop_tolr,
        expert_factor=expert_factor,
        logger=logger
    )

    # -- CONVERT ORDERED OBJECTS TO UNORDERED
    # Map parameter names back onto points
    x_hat_df = pd.DataFrame(data=x_hat, index=parameter_order, columns=["x_hat"]).T

    return x_hat_df, f_hat

Since we know that our parameter space is not large, we can log with two levels of debugging information and use the logs to check the correctness of points.  This also provides us with more detailed method logging details.  The contents of our example JSON configuration file are

In [6]:
# Remove input from rendering

import json

JSON_FILENAME = Path.cwd().joinpath("test.json")

TEST_CFG = {
    "starting_point": {"alpha": 1.1, "beta": -2.2},
    "max_iters": 10,
    "stopping_criterion": 1.0e-2,
    "expert": {
        "factor": 0.25
    },
    "logging": {
        "Level": poptus.LOG_LEVEL_MIN_DEBUG + 1
    }
}

with open(JSON_FILENAME, "w") as fptr:
    json.dump(TEST_CFG, fptr, indent=4)

print_file(JSON_FILENAME)

{
    "starting_point": {
        "alpha": 1.1,
        "beta": -2.2
    },
    "max_iters": 10,
    "stopping_criterion": 0.01,
    "expert": {
        "factor": 0.25
    },
    "logging": {
        "Level": 3
    }
}


Note that we have specified our own expert configuration value to achieve more aggressive "convergence."

In [7]:
x_hat_df, f_hat = run_method(JSON_FILENAME, PARAMETER_ORDER)
x_hat_df

[MyMethod] Configuration loaded from /Users/jared/Projects/POptUS/POptUS_public/book/notebooks/test.json
[MyMethod] 
[MyMethod] Parameter-space Basis
[MyMethod] --------------------------------------------------------------------------------
[MyMethod] e_1       beta
[MyMethod] e_2       alpha
[MyMethod] 
[MyMethod] Method Configuration
[MyMethod] --------------------------------------------------------------------------------
[MyMethod] N Max Iterations              10
[MyMethod] Stopping Tolerance            0.01
[MyMethod] Expert Configurations
[MyMethod] 	Factor                   0.25
[MyMethod] 
[MyMethod] Starting Point
[MyMethod] ------------------------------------------------------------
[MyMethod] 	e_1       -2.2
[MyMethod] 	e_2       1.1
[MyMethod] 
[MyMethod] WARNING - Stopping tolerance is suspiciously large
[MyMethod] f_1 = 2.459674775249769 at [-2.2  1.1]
[MyMethod] f_2 = 0.6149186938124422 at [-0.55   0.275]
[MyMethod] f_3 = 0.15372967345311056 at [-0.1375   0.06875]
[M

,beta,alpha
x_hat,-0.008594,0.004297
